# Sam Jackson, MAE 494, HW3

This code provides answers to the questions posed in Homework 3. The packages used in this homework assignment are numpy, matplotlib.pyplot and scipy.optimize.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

a